## Adding new flavors to a `RailProject` and using them

This notebook will show you the basics of adding new flavors to `RailProject` and using them.  It's quite simple.

#### Import the usual suspects

In [ ]:
import os
from rail.projects import RailProject, RailFlavor, library

#### Setup environment, and move to correct directory.  This will also download the needed data files to run this example.

In [ ]:
import os
from rail.projects import library

check_dir = os.path.basename(os.path.abspath(os.curdir))
if check_dir == 'examples':
    os.chdir('..')

print(os.path.abspath(os.curdir))

setup = library.setup_mininal_example_files()
assert setup == 0

#### Build a very minimal project, with only the baseline flavor

In [ ]:
project = RailProject.load_config('tests/minimal.yaml')

#### Add a new flavor to the project

Note that we are specifying:

1. a name for the new flavor
2. the pipelines that we can run under the new flavor
3. overrides for the pipelines with respect to the baseline

In [ ]:
new_flavor = project.add_flavor(
    name='new_flavor', 
    pipelines=['pz', 'tomography'],
    pipeline_overrides=dict(
        default=dict(
            kwargs=dict(
              algorithms=['gpz'],
            )
        ),
        pz=dict(
            inform_gpz=dict(
              gpz_method='VC',
            )
        )
    )
)


#### Make sure the flavor got added, and inspect it's configuration. 

Note that it picked up a few parameters from the baseline

In [ ]:
project.get_flavors()

In [ ]:
new_flavor.config

Let's see where the pipeline files will go

In [ ]:
print(f"The template for pipeline files is: {project.get_path('pipeline_path')}")

In [ ]:
print(f"The path for the 'pz' pipeline of flavor 'baseline' is {project.get_path('pipeline_path', pipeline='pz', flavor='baseline')}")

#### Build the pipelines for the few flavor

In [ ]:
project.build_pipelines('new_flavor', force=True)

#### Let's see what the inputs for the new pz pipeline are

In [ ]:
pipeline_info = project.get_pipeline('pz')
pipeline_instance = pipeline_info.make_instance(project, 'new_flavor', {})
pipeline_instance.get_input_files(project, selection='gold')

#### All set, run the pipeline

In [ ]:
project.run_pipeline_single('pz', flavor='new_flavor', selection='gold')